In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline  ###管道流
# from sklearn.neighbors import KNeighborsClassifier  #k近邻(knn)算法
# from sklearn.linear_model import LogisticRegression
# 构建lgb模型
from lightgbm import LGBMRegressor
import warnings  #去除警告包
warnings.filterwarnings('ignore')#去除警告
from sklearn.metrics import mean_squared_error


In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [3]:
### 一般我们机器学习的流程

# 一、加载数据
#processs_datas/train_process_datas_x.csv
datas_x = pd.read_csv('./processs_datas/train_process_datas_x.csv')
# print(datas_x.head())
# print(datas_x.info(verbose=True,null_counts=True))
# processs_datas/train_datas_Y.csv
datas_y = pd.read_csv('./processs_datas/train_datas_Y.csv')
# print(datas_y.head())
# print(datas_y.info())
# print(datas_y['happiness'].value_counts())

### 标签里面存在异常值 -8
# ## 一般情况下对于标签存在异常的数据需要删除，注意对应的X也要删除
# ## 先合并X,Y,再删除
# datas = pd.concat([datas_x,datas_y],axis=1)
# # print(datas.head())
# # print(datas.info(verbose=True,null_counts=True))
# datas.replace(-8,np.nan,inplace=True)
# datas.dropna(axis=0,how='any',inplace=True)
# # print(datas.shape)
# # print(datas['happiness'].value_counts(dropna=True))

## 这里我们可以考虑将-8替换成3（特殊情况特殊对待）
datas_y.replace(-8,3,inplace=True)
# print(datas_y.shape)
# print(datas_y['happiness'].value_counts())

In [4]:
# 二、数据清洗与预处理  （这里在dataprocess里面完成了）
# 删除ID字段
datas_x.drop(['id'],axis=1,inplace=True)

# 三、获取数据的特征属性X和目标属性Y

# 四、数据分割
x_train,x_test,y_train,y_test = train_test_split(datas_x,datas_y,test_size=0.2,random_state=11)
# print(x_train.shape)
# print(x_test.shape)

In [5]:
# # 五、特征工程  这里我们做个PCA降维 ## 使用管道流

# # 六、模型构建
# ##我们这里使用网格交叉验证,LGB回归（使用回归：评估指标是mse）
# # # 网格调参
# params ={
#           'n_estimators': [400,700,1000],
#           'max_depth':  [3, 5, 7],
#           'num_leaves': [10,15,20],
#           'subsample': [0.6, 0.8, 1.0],
#           'colsample_bytree': [0.2,0.6,0.8],
#           'learning_rate' : [0.01,0.02,0.03]
# }

# clf=GridSearchCV(estimator=LGBMRegressor(),param_grid=params,cv=3)

# clf.fit(datas_x,datas_y)
# print('最好的模型：',clf.best_estimator_)
# print('最好模型的评分：',clf.best_score_)
# print('最优参数：',clf.best_params_)
# params_dir_path = './model'
# import os
# if os.path.exists(save_dir_path):
#     pass
# else:
#     os.makedirs(save_dir_path)
# with open('../model2/lgbparams.txt','w',encoding='utf-8') as file:
#     file.write('最优参数：{} \n 最好模型的评分：{}'.format(clf.best_params_,clf.best_score_))


In [6]:
lgb = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.2,
       importance_type='split', learning_rate=0.01, max_depth=5,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=1000, n_jobs=-1, num_leaves=20, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=0.6, subsample_for_bin=200000, subsample_freq=0
                     )


lgb.fit(x_train,y_train)
print(lgb.score(x_train,y_train))
print(lgb.score(x_test,y_test))

y_train_hat=lgb.predict(x_train)
y_test_hat=lgb.predict(x_test)
print('训练集mse为:',mean_squared_error(y_train,y_train_hat))
print('测试集mse为:',mean_squared_error(y_test,y_test_hat))

0.5368827699335841
0.3283652097480225
训练集mse为: 0.310856122802509
测试集mse为: 0.4478292872702622


In [7]:
# 九、模型持久化
## 实际上对于最终我们保存的模型来说，我们可以把所有数据进行训练，再保存模型
lgb = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.2,
              importance_type='split', learning_rate=0.02, max_depth=7,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=700, n_jobs=-1, num_leaves=20, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=0.6, subsample_for_bin=200000, subsample_freq=0
                     )
lgb.fit(datas_x,datas_y)
print(lgb.score(datas_x,datas_y))
datas_y_hat = lgb.predict(datas_x)
print(mean_squared_error(datas_y,datas_y_hat))
import joblib

save_dir_path = './model'
import os
if os.path.exists(save_dir_path):
    pass
else:
    os.makedirs(save_dir_path)

joblib.dump(lgb,save_dir_path+'/lgb_happiness.pkl')

0.5777518679410373
0.2830481700352664


['./model/lgb_happiness.pkl']

In [8]:
lgb.feature_importances_

array([ 22, 163, 332, 335,  79,  40,  53,  36,  68,  62, 208,  20, 248,
         4,  12,  29,  39,  12,   0,   0,   0,  12, 182, 235, 222, 107,
       304,  32,  61,  52,  38,  81, 101,  49,  45,  82,  38,  90,  48,
        30,  91,  58, 117, 126,  18,  46,  41,  62, 109,  53, 171, 114,
        67, 328, 207, 146, 189, 153,  73,  63,  92,  24,  42,  17,  34,
        11,  10, 235, 145, 182, 130,  47,   2,  17,  18,   4,   2,   0,
         0,   0,   0,  59,  93,  50,  80, 104,  28,  56, 165,  54,  47,
        15,  55,  26,  94,  64,  61,  64,  86, 125, 137, 175, 199,  83,
        79,  80, 103,  78,  89,  66,  85,  91, 100,  58,  94,  56, 179,
       150, 149, 150, 199, 176, 200, 176, 226, 169,   0, 143,  88, 491,
        20, 139, 172, 260, 124, 165, 155,  39])